In [12]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests
from config import weather_api_key
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
from datetime import datetime

In [13]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [14]:
# Add the latitudes and longitudes to a list. unpack our lat_lngs zip object into a list
coordinates = list(lat_lngs)

In [15]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

602

In [16]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
len(city_data)

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 2 of Set 1 | grand centre
City not found. Skipping...
Processing Record 3 of Set 1 | deputatskiy
Processing Record 4 of Set 1 | constitucion
Processing Record 5 of Set 1 | airai
Processing Record 6 of Set 1 | santa maria
Processing Record 7 of Set 1 | khandyga
Processing Record 8 of Set 1 | rikitea
Processing Record 9 of Set 1 | yarmouth
Processing Record 10 of Set 1 | ashland
Processing Record 11 of Set 1 | sentyabrskiy
City not found. Skipping...
Processing Record 12 of Set 1 | kahului
Processing Record 13 of Set 1 | dikson
Processing Record 14 of Set 1 | bluff
Processing Record 15 of Set 1 | mahebourg
Processing Record 16 of Set 1 | provideniya
Processing Record 17 of Set 1 | gold coast
Processing Record 18 of Set 1 | derzhavinsk
Processing Record 19 of Set 1 | belushya guba
City not found. Skipping...
Processing Record 20 of Set 1 | cidr

Processing Record 39 of Set 4 | ukiah
Processing Record 40 of Set 4 | cairns
Processing Record 41 of Set 4 | joao camara
Processing Record 42 of Set 4 | bambous virieux
Processing Record 43 of Set 4 | tigil
Processing Record 44 of Set 4 | bengkulu
Processing Record 45 of Set 4 | itarema
Processing Record 46 of Set 4 | balasheyka
Processing Record 47 of Set 4 | omboue
Processing Record 48 of Set 4 | los llanos de aridane
Processing Record 49 of Set 4 | attawapiskat
City not found. Skipping...
Processing Record 50 of Set 4 | bourail
Processing Record 1 of Set 5 | ampanihy
Processing Record 2 of Set 5 | yellowknife
Processing Record 3 of Set 5 | antalaha
Processing Record 4 of Set 5 | toila
Processing Record 5 of Set 5 | tamworth
Processing Record 6 of Set 5 | kathu
Processing Record 7 of Set 5 | lolua
City not found. Skipping...
Processing Record 8 of Set 5 | kimberley
Processing Record 9 of Set 5 | vaitupu
City not found. Skipping...
Processing Record 10 of Set 5 | saint-philippe
Proces

Processing Record 27 of Set 8 | tomatlan
Processing Record 28 of Set 8 | selma
Processing Record 29 of Set 8 | aksarka
Processing Record 30 of Set 8 | norman wells
Processing Record 31 of Set 8 | vila franca do campo
Processing Record 32 of Set 8 | chicama
Processing Record 33 of Set 8 | toktogul
Processing Record 34 of Set 8 | sao miguel
Processing Record 35 of Set 8 | tigzirt
Processing Record 36 of Set 8 | olafsvik
Processing Record 37 of Set 8 | mino
Processing Record 38 of Set 8 | sault sainte marie
Processing Record 39 of Set 8 | aklavik
Processing Record 40 of Set 8 | hvide sande
Processing Record 41 of Set 8 | celendin
Processing Record 42 of Set 8 | karakol
Processing Record 43 of Set 8 | dolbeau
City not found. Skipping...
Processing Record 44 of Set 8 | goderich
Processing Record 45 of Set 8 | kruisfontein
Processing Record 46 of Set 8 | timmins
Processing Record 47 of Set 8 | lorengau
Processing Record 48 of Set 8 | yining
Processing Record 49 of Set 8 | ketchikan
Processin

Processing Record 18 of Set 12 | carlyle
Processing Record 19 of Set 12 | flin flon
Processing Record 20 of Set 12 | pietarsaari
Processing Record 21 of Set 12 | saint-joseph
Processing Record 22 of Set 12 | palora
Processing Record 23 of Set 12 | nishihara
Processing Record 24 of Set 12 | aligudarz
Processing Record 25 of Set 12 | filingue
Processing Record 26 of Set 12 | albina
Processing Record 27 of Set 12 | krotoszyn
Processing Record 28 of Set 12 | kirakira
Processing Record 29 of Set 12 | pingliang
Processing Record 30 of Set 12 | pimentel
Processing Record 31 of Set 12 | yilan
Processing Record 32 of Set 12 | moldova noua
Processing Record 33 of Set 12 | nikolayevka
Processing Record 34 of Set 12 | san policarpo
Processing Record 35 of Set 12 | husavik
Processing Record 36 of Set 12 | soissons
Processing Record 37 of Set 12 | ereymentau
Processing Record 38 of Set 12 | dujuma
City not found. Skipping...
Processing Record 39 of Set 12 | beidao
Processing Record 40 of Set 12 | as

558

In [17]:
len(city_data)

558

In [18]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Deputatskiy,69.3000,139.9000,1.36,83,77,1.12,RU,2021-04-07 04:40:29
1,Constitucion,-35.3333,-72.4167,55.51,87,82,4.59,CL,2021-04-07 04:40:29
2,Airai,-8.9266,125.4092,71.01,74,92,2.53,TL,2021-04-07 04:40:29
3,Santa Maria,-29.6842,-53.8069,64.42,88,3,5.30,BR,2021-04-07 04:40:30
4,Khandyga,62.6667,135.6000,9.84,76,100,2.59,RU,2021-04-07 04:40:30
5,Rikitea,-23.1203,-134.9692,77.20,67,18,12.24,PF,2021-04-07 04:40:30
6,Yarmouth,41.7057,-70.2286,46.40,71,75,3.44,US,2021-04-07 04:40:31
7,Ashland,40.8167,-82.2665,63.00,62,50,8.84,US,2021-04-07 04:37:33
8,Kahului,20.8947,-156.4700,73.40,56,40,13.80,US,2021-04-07 04:38:57
9,Dikson,73.5069,80.5464,4.64,90,60,7.11,RU,2021-04-07 04:40:31


In [19]:
# Reorder the column order
new_column_order = ["City", "Country", "Date", "Lat","Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]

city_data_df = city_data_df[new_column_order]

city_data_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Deputatskiy,RU,2021-04-07 04:40:29,69.3000,139.9000,1.36,83,77,1.12
1,Constitucion,CL,2021-04-07 04:40:29,-35.3333,-72.4167,55.51,87,82,4.59
2,Airai,TL,2021-04-07 04:40:29,-8.9266,125.4092,71.01,74,92,2.53
3,Santa Maria,BR,2021-04-07 04:40:30,-29.6842,-53.8069,64.42,88,3,5.30
4,Khandyga,RU,2021-04-07 04:40:30,62.6667,135.6000,9.84,76,100,2.59
...,...,...,...,...,...,...,...,...,...
553,San Francisco,US,2021-04-07 04:41:12,37.7749,-122.4194,51.01,83,45,3.00
554,Camargo,MX,2021-04-07 04:43:35,27.6667,-105.1667,74.16,16,62,9.80
555,Sretensk,RU,2021-04-07 04:43:35,52.2453,117.7093,45.90,34,17,5.79
556,Tessalit,ML,2021-04-07 04:43:36,20.1986,1.0114,72.88,7,0,5.93


In [22]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# Extract relevant fields from the DataFrame for plotting.
lats = city_data_df["Lat"]
max_temps = city_data_df["Max Temp"]
humidity = city_data_df["Humidity"]
cloudiness = city_data_df["Cloudiness"]
wind_speed = city_data_df["Wind Speed"]